## Importación de librerias

In [1]:
import mysql.connector

In [2]:
import pandas as pd

In [38]:
import pingouin as pgn
from bokeh.plotting import show,figure

In [40]:
from bokeh.layouts import layout

In [12]:
from scipy import stats
import math
import numpy as np

In [15]:
import random

## Conecciones

In [5]:
try:
    connection=mysql.connector.connect(
        host='192.168.1.200',
        port=3306,
        user='remoteuser',
        password='1234',
        db='Packed_company'
    )
    cursor=connection.cursor()
    cursor.execute("""SELECT SAMPLE_RIVER_DOWN
            FROM tbl_samplemeasure""")
    answer=cursor.fetchall()
    df=pd.DataFrame(answer)
except Exception as ex:
    raise Exception(ex)
    
    

## Regresion lineal 2

In [6]:
def lineal_regression_v2(data,column1,column2,widht=400,heigth=400):

    #Calculo de la regresion lineal.
    rls=pgn.linear_regression(X=data[column1],y=data[column2],as_dataframe=False)
    b0=rls['coef'][0]
    b1=rls['coef'][1]

    data['Y_Estimada']=b0+b1*data[column1]
    
    
    #Calculo de la media y el numero de datos en la columna.
    x_media=(data[column1].mean())
    n=len(data[column1])
    #Calculo del estadistico t para los intervalos de confianza y predicción respectivamente.
    alpha=0.05
    stadistic=abs(stats.t.ppf(alpha/2,df=n-2))
    #Calculo de Sxx
    data['Error']=+data[column2]-data['Y_Estimada']
    data['Error_Sqr']=+(data['Error'])**2
    s_sqr=(data['Error_Sqr'].sum())/(n-2)
    s=s_sqr**0.5
    data['xx']=(data[column1]-x_media)**2
    #Constante
    Sxx=data['xx'].sum()
    
    data['LIPI']=data['Y_Estimada']-stadistic*s*((1/n)+((data[column1]-x_media)**2/Sxx))**0.5
    data['LSPI']=data['Y_Estimada']+stadistic*s*((1/n)+((data[column1]-x_media)**2/Sxx))**0.5

    data['LIPIF']=data['Y_Estimada']-stadistic*s*(1+(1/n)+((data[column1]-x_media)**2/Sxx))**0.5
    data['LSPIF']=data['Y_Estimada']+stadistic*s*(1+(1/n)+((data[column1]-x_media)**2/Sxx))**0.5


    min_value=int(data[column1].min())-1
    max_value=int(data[column1].max())+1
    #Ecuación de regresión lineal simple
    x=[i for i in range(min_value,max_value,1)]
    Y_estimada=[(b0+b1*j) for j in x]
    
    #Ecuación de limite infererior y superior de predicción inside (lipi & lspi)
    LIPI=[(Y_estimada[i]-stadistic*s*((1/n)+((x[i]-x_media)**2/Sxx))**0.5) for i in range(len(x))]
    LSPI=[(Y_estimada[i]+stadistic*s*((1/n)+((x[i]-x_media)**2/Sxx))**0.5) for i in range(len(x))]

    #Ecuación de limite infererior y superior de predicción future (lipif & lspif)
    LIPIF=[(Y_estimada[j]-stadistic*s*(1+(1/n)+((x[j]-x_media)**2/Sxx))**0.5) for j in range(len(x))]
    LSPIF=[(Y_estimada[j]+stadistic*s*(1+(1/n)+((x[j]-x_media)**2/Sxx))**0.5) for j in range(len(x))]
    
    
    figure1=figure(
        title=column2,
        plot_width=widht,plot_height=heigth,
        toolbar_location='below')
    
    figure1.xgrid.grid_line_color=None
    figure1.xaxis.axis_label=column1
    figure1.xaxis.major_label_text_font_size='15px'
    figure1.yaxis.axis_label=column2
    figure1.toolbar.autohide=True

    figure1.scatter(data[column1],data[column2],size=2,color='navy',alpha=0.5)
    figure1.line(x,Y_estimada)
    figure1.line(x,LIPI)
    figure1.line(x,LSPI)
    figure1.line(x,LSPIF)
    figure1.line(x,LIPIF)
    figure1.varea(x=x,
                  y1=LSPI,
                  y2=LIPI,fill_color='#0CE82A',alpha=0.3)
    
    figure1.varea(x=x,
                  y1=LSPIF,
                  y2=LIPIF,fill_color='#0C70E8',alpha=0.3)
    
    return figure1

In [10]:
def lineal_regression_v2(data,column1,column2,variable):

    #Calculo de la regresion lineal.
    rls=pgn.linear_regression(X=data[column1],y=data[column2],as_dataframe=False)
    b0=rls['coef'][0]
    b1=rls['coef'][1]

    data['Y_Estimada']=b0+b1*data[column1]
    
    
    #Calculo de la media y el numero de datos en la columna.
    x_media=(data[column1].mean())
    n=len(data[column1])
    #Calculo del estadistico t para los intervalos de confianza y predicción respectivamente.
    alpha=0.05
    stadistic=abs(stats.t.ppf(alpha/2,df=n-2))
    #Calculo de Sxx
    data['Error']=+data[column2]-data['Y_Estimada']
    data['Error_Sqr']=+(data['Error'])**2
    s_sqr=(data['Error_Sqr'].sum())/(n-2)
    s=s_sqr**0.5
    data['xx']=(data[column1]-x_media)**2
    #Constante
    Sxx=data['xx'].sum()
    
    data['LIPI']=data['Y_Estimada']-stadistic*s*((1/n)+((data[column1]-x_media)**2/Sxx))**0.5
    data['LSPI']=data['Y_Estimada']+stadistic*s*((1/n)+((data[column1]-x_media)**2/Sxx))**0.5

    data['LIPIF']=data['Y_Estimada']-stadistic*s*(1+(1/n)+((data[column1]-x_media)**2/Sxx))**0.5
    data['LSPIF']=data['Y_Estimada']+stadistic*s*(1+(1/n)+((data[column1]-x_media)**2/Sxx))**0.5


    min_value=int(data[column1].min())-1
    max_value=int(data[column1].max())+1
    #Ecuación de regresión lineal simple
    x=variable
    Y_estimada=(b0+b1*x)
    
    #Ecuación de limite infererior y superior de predicción inside (lipi & lspi)
    LIPI=(Y_estimada-stadistic*s*((1/n)+((x-x_media)**2/Sxx))**0.5)
    LSPI=(Y_estimada+stadistic*s*((1/n)+((x-x_media)**2/Sxx))**0.5)

    #Ecuación de limite infererior y superior de predicción future (lipif & lspif)
    LIPIF=(Y_estimada-stadistic*s*(1+(1/n)+((x-x_media)**2/Sxx))**0.5)
    LSPIF=(Y_estimada+stadistic*s*(1+(1/n)+((x-x_media)**2/Sxx))**0.5)
    data={
        "Y_estimada":Y_estimada,
        "LIPI":LIPI,
        "LSPI":LSPI,
        "LIPIF":LIPIF,
        "LSPIF":LSPIF }
    
    return data

## Histograma

In [6]:
def histogram_plot(Data,column,name,distribution,m=0,std=1,s=0.9,n=500,p=0.1,ancho=400,alto=400):
    N=Data[column].count()
    figure_h=figure(title=name,plot_width=ancho,plot_height=alto,
               toolbar_location="below")
    figure_h.xgrid.grid_line_color=None
    figure_h.ygrid.grid_line_alpha=0.7
    figure_h.xaxis.axis_label=str(column)
    figure_h.yaxis.axis_label='Probability'
    figure_h.toolbar.autohide=True
    Number_intervals=int(round(1+3.322*math.log10(N),0))
    
    if distribution=='normal':
        hist,edges=np.histogram(Data[column],density=True,bins=Number_intervals)
        x = np.linspace(min(Data[column]), max(Data[column]), num=N)
        y = stats.norm.pdf(x,m,std)
        # Histograma distribución normal
        figure_h.quad(top=hist,bottom=0,left=edges[:-1],right=edges[1:],
              fill_color='#008080',line_color='black',legend_label='Normal_distribution')
        figure_h.line(x,y,color='#181515',line_width=2)
        return figure_h 
        
    elif distribution=='lognormal':
        hist,edges=np.histogram(Data[column],density=True,bins=100)
        x = np.linspace(min(Data[column]), max(Data[column]), num=N)
        y = stats.lognorm.pdf(x,s)
        #Histograma distribucion logonormal
        figure_h.quad(top=hist,bottom=0,left=edges[:-1],right=edges[1:],
              fill_color='#008080',line_color='black',legend_label='Logormal_distribution')
        figure_h.line(x,y,color='#181515',line_width=2)
        return figure_h
        
    elif distribution=='binomial':
        hist,edges=np.histogram(Data[column],density=True,bins=Number_intervals)
        x = np.linspace(min(Data[column]), max(Data[column]), num=N)
        y = stats.binom.pmf(x,n,p)
        #Histograma distribucion binomial
        figure_h.quad(top=hist,bottom=0,left=edges[:-1],right=edges[1:],
              fill_color='#008080',line_color='black',legend_label='Binomial_distribution')
        figure_h.line(x,y,color='#181515',line_width=2)
        return figure_h


## Prueba de funciones

In [7]:
df=df.rename(columns={0:'x',1:'y'})


In [8]:
df

,x
0,0.298581
1,0.204635
2,0.281319
3,0.294748
4,0.178016
...,...
495,0.223788
496,0.127096
497,0.161890
498,0.247018


In [13]:
#histogram_plot(Data,column,name,distribution,m=0,std=1,s=0.9,n=500,p=0.1,ancho=400,alto=400)
mean=df['x'].mean()
stdev=df['x'].std()
histogram=histogram_plot(df,'x','normality_test','normal',m=mean,std=stdev)

In [14]:
show(histogram)

In [16]:
#Random.sample(list,k)
#tomara una muestra de valores random del conjunto de mediciones que tenemos

In [19]:
#Definir el tamaño de muestra de una poblacion.
p=0.5
z=1.96
e=0.05
N=df['x'].count()
n=(N*z**2)*p*(1-p)/((N-1)*e+(z**2)*p*(1-p))

18.533098678522904

In [56]:
#Primeros 100 valores
a=[np.array(random.sample(list(df['x']),k=int(3))).mean() for i in range(100)]
data={
    "x":a
}
df_2=pd.DataFrame(data)
histogram1=histogram_plot(df_2,'x','normality_test','normal',m=df_2['x'].mean(),std=df_2['x'].std())


In [57]:
#Primeros 200 valores
a=[np.array(random.sample(list(df['x']),k=int(5))).mean() for i in range(200)]
data={
    "x":a
}
df_2=pd.DataFrame(data)
histogram2=histogram_plot(df_2,'x','normality_test','normal',m=df_2['x'].mean(),std=df_2['x'].std())


In [58]:
#Primeros 300 valores
a=[np.array(random.sample(list(df['x']),k=int(10))).mean() for i in range(300)]
data={
    "x":a
}
df_2=pd.DataFrame(data)
histogram3=histogram_plot(df_2,'x','normality_test','normal',m=df_2['x'].mean(),std=df_2['x'].std())


In [59]:
#Primeros 500 valores
a=[np.array(random.sample(list(df['x']),k=int(15))).mean() for i in range(500)]
data={
    "x":a
}
df_2=pd.DataFrame(data)
histogram4=histogram_plot(df_2,'x','normality_test','normal',m=df_2['x'].mean(),std=df_2['x'].std())


In [60]:
#Primeros 1000 valores
a=[np.array(random.sample(list(df['x']),k=int(round(n,0)))).mean() for i in range(1000)]
data={
    "x":a
}
df_2=pd.DataFrame(data)
histogram5=histogram_plot(df_2,'x','normality_test','normal',m=df_2['x'].mean(),std=df_2['x'].std())


In [61]:
layout1=layout([histogram1,histogram2,histogram3,histogram4,histogram5])
show(layout1)

In [ ]:
def central_limit(df):
    a=[np.array(random.sample(list(df['x']),k=int(3))).mean() for i in range(100)]
    data={
        "x":a
    }
    df_2=pd.DataFrame(data)
    histogram1=histogram_plot(df,'x','normality_test','normal',m=df['x'].mean(),std=df['x'].std())
    